In [ ]:
# Make sure you have pymc3 installed
import sys
!{sys.executable} -m pip install pymc3

In [ ]:
# make sure the notebook reloads the module each time we modify it
%load_ext autoreload
%autoreload 2

# make sure the displays are nice
%matplotlib notebook
#figsize(12,8)

In [ ]:
import parametric_regression_student_version as pr
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks")
import numpy as np
import pymc3 as pm

## Preparing data and utilities

In [ ]:
X, y, theta_true, sigma_noise, indices_support = pr.generate_data(sample_size = 50, dimension = 100, seed = 1)
plt.plot(X[:,8], y, '+')
plt.show()

In [ ]:
fake_std = .5*np.ones(theta_true.shape)
fake_theta_hat = np.zeros(theta_true.shape)
fig, ax = plt.subplots(2, figsize=(10,8))
pr.plot_coefficients(ax, theta_true=theta_true, indices_support=indices_support, theta_hat=fake_theta_hat, lower_bound=fake_theta_hat-fake_std, 
                     upper_bound=fake_theta_hat+fake_std)
plt.show()

## Using sklearn's lasso implementation as baseline

In [ ]:
theta_lasso = pr.get_sklearn_lasso_estimate(X, y)
print(theta_lasso.shape)

In [ ]:
fig, ax = plt.subplots(2, figsize=(10,6))
pr.plot_coefficients(ax, theta_true=theta_true, indices_support=indices_support, theta_hat=theta_lasso, label="lasso")
plt.show()

## Bayesian lasso using a Laplace prior

In [ ]:
trace = pr.get_mcmc_sample_for_laplace_prior(X, y)
accept = np.sum(trace['theta'][1:,0] != trace['theta'][:-1,0])
print("Acceptance Rate: ", accept/trace['theta'][:,0].shape[0])

In [ ]:
pm.summary(trace)

In [ ]:
score=pm3.geweke(trace, first=0.1, last=0.5, intervals=20)
plt.scatter(score[0]['Mean of Data'][:,0],score[0]['Mean of Data'][:,1], marker = 'o', s=100)
plt.axhline(-1.98, c='r')
plt.axhline(1.98, c='r')
plt.ylim(-2.5,2.5)
plt.xlim(0-10,.5*trace['Mean of Data'].shape[0]/2+10)
plt.title('Geweke Plot Comparing first 10% and Slices of the Last 50% of Chain\nDifference in Mean Z score')
plt.show()

In [ ]:
theta_laplace = trace.get_values('theta').mean(0) # not expected to show much sparsity
lower_bound_laplace = np.quantile(trace.get_values('theta'), 0.025, axis=0)
upper_bound_laplace = np.quantile(trace.get_values('theta'), 0.975, axis=0)

In [ ]:
#pm.traceplot(trace)

In [ ]:
f, ax = plt.subplots(2, figsize=(10,8))
pr.plot_coefficients(ax, theta_true, indices_support=indices_support, theta_hat=theta_laplace, lower_bound=lower_bound_laplace, upper_bound=upper_bound_laplace,
             color="blue", label="laplace")
pr.plot_coefficients(ax, theta_true, indices_support=indices_support, theta_hat=theta_lasso, color="red", label="lasso")
plt.show()

The Laplace prior tends to overregularize. Can you get rid of that problem playing with the prior?

## Bayesian lasso using the horseshoe prior

In [ ]:
trace_horseshoe = pr.get_mcmc_sample_for_horseshoe_prior(X, y)

In [ ]:
pm.summary(trace_horseshoe)

In [ ]:
print(pm.gelman_rubin(trace_horseshoe))

In [ ]:
theta_horseshoe = trace_horseshoe.get_values('theta').mean(0)
lower_bound_horseshoe = np.quantile(trace_horseshoe.get_values('theta'), 0.025, axis=0)
upper_bound_horseshoe = np.quantile(trace_horseshoe.get_values('theta'), 0.975, axis=0)

In [ ]:
f, ax = plt.subplots(2, figsize=(10,8))
pr.plot_coefficients(ax, theta_true, indices_support=indices_support, theta_hat=theta_laplace, lower_bound=lower_bound_laplace, upper_bound=upper_bound_laplace,
                     color="blue", label="laplace")
pr.plot_coefficients(ax, theta_true=theta_true, indices_support=indices_support, theta_hat=theta_horseshoe, lower_bound=lower_bound_horseshoe, 
                     upper_bound=upper_bound_horseshoe, color="orange", label="horseshoe")
#pr.plot_coefficients(theta_true, theta_lasso, color="red", label="lasso")

plt.show()

## Bayesian lasso using the Finnish horseshoe prior

In [ ]:
trace_finnish_horseshoe = pr.get_mcmc_sample_for_finnish_horseshoe_prior(X, y)

In [ ]:
theta_finnish_horseshoe = trace_finnish_horseshoe.get_values('theta').mean(0)
lower_bound_finnish_horseshoe = np.quantile(trace_finnish_horseshoe.get_values('theta'), 0.025, axis=0)
upper_bound_finnish_horseshoe = np.quantile(trace_finnish_horseshoe.get_values('theta'), 0.975, axis=0)

In [ ]:
f, ax = plt.subplots(2, figsize=(10,8))
pr.plot_coefficients(ax, theta_true, indices_support=indices_support, theta_hat=theta_laplace,
                     color="blue", label="laplace")
pr.plot_coefficients(ax, theta_true=theta_true, indices_support=indices_support, theta_hat=theta_horseshoe, lower_bound=lower_bound_horseshoe, 
                     upper_bound=upper_bound_horseshoe, color="orange", label="horseshoe")
pr.plot_coefficients(ax, theta_true, indices_support=indices_support, theta_hat=theta_finnish_horseshoe, lower_bound=lower_bound_finnish_horseshoe, 
                     upper_bound=upper_bound_finnish_horseshoe, color="magenta", label="finnish")
#pr.plot_coefficients(theta_true, theta_lasso, color="red", label="lasso")

plt.show()